In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `Dissertation` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenti

In [ ]:
!pip install llama-cpp-python langchain sentence-transformers openai==0.28.0 transformers torch "accelerate>=0.26.0" faiss-cpu ipywidgets gradio



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 MB 30.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Set the base path to your data files.
# Update this if your folder name is different.
BASE_PATH = '/content/drive/MyDrive/ColabNotebooks/Dissertataion'
print("BASE_PATH is set to:", BASE_PATH)


In [ ]:
import glob
import json
import os
import time
import pickle
import logging
from tqdm import tqdm
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

logging.basicConfig(
    filename="faiss_errors.log",
    filemode="a",
    format="%(asctime)s - %(levelname)s - %(message)s",
    level=logging.ERROR
)

print("All packages imported and logging configured!")
print("NumPy version:", np.__version__)


All packages imported and logging configured!
NumPy version: 2.0.2


In [ ]:
# Ensure BASE_PATH matches the folder containing your files:
BASE_PATH = '/content/drive/MyDrive/ColabNotebooks/Dissertation'
print("BASE_PATH is set to:", BASE_PATH)

# Load FAISS index:
index = faiss.read_index(os.path.join(BASE_PATH, "my_index.idx"))
print("FAISS index loaded from disk.")

# Load metadata files:
with open(os.path.join(BASE_PATH, "all_texts.pkl"), "rb") as f:
    all_texts = pickle.load(f)
with open(os.path.join(BASE_PATH, "doc_ids.pkl"), "rb") as f:
    doc_ids = pickle.load(f)
with open(os.path.join(BASE_PATH, "chunk_nums.pkl"), "rb") as f:
    chunk_nums = pickle.load(f)
print("Metadata loaded from pickle files.")


BASE_PATH is set to: /content/drive/MyDrive/ColabNotebooks/Dissertation
FAISS index loaded from disk.
Metadata loaded from pickle files.


In [ ]:
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("Embedding model loaded.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model loaded.


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Testing using the pipeline helper:
test_pipe = pipeline("text-generation",
                       model="meta-llama/Llama-3.2-1B-Instruct",
                       tokenizer="meta-llama/Llama-3.2-1B-Instruct")
test_output = test_pipe("Who are you?", max_new_tokens=32, do_sample=True)
print("Test output from the 1B model (pipeline):")
print(test_output[0]["generated_text"])

# Alternatively, load the model directly (if you need more control):
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
llama_model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-1B-Instruct")
print("Direct loading of the Llama 1B model successful!")


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Test output from the 1B model (pipeline):
Who are you? What do you do?
I'm a 35-year-old woman who works as a marketing manager for a small business. I love my job, but sometimes it
Direct loading of the Llama 1B model successful!


In [ ]:
def generate_answer(user_query):
    # Generate an embedding for the user query using the embedding model
    query_embedding = embedding_model.encode([user_query]).astype("float32")

    # Retrieve the top k similar texts from the FAISS index
    k = 5  # Number of similar texts to retrieve
    distances, indices = index.search(query_embedding, k)

    print("FAISS indices:", indices)
    print("FAISS distances:", distances)

    # Retrieve texts using valid indices
    retrieved_texts = []
    for idx in indices[0]:
        if 0 <= idx < len(all_texts):
            retrieved_texts.append(all_texts[idx])
        else:
            print(f"Warning: Index {idx} is out of bounds for all_texts with length {len(all_texts)}.")

    if not retrieved_texts:
        return "No valid texts retrieved from the index."

    # Combine the retrieved texts as context
    context = "\n".join(retrieved_texts)

    # Construct an augmented query with instructions not to restate the context
    few_shot_example = (
        "Example 1:\n"
        "Q: What are the key challenges in unsupervised domain adaptation in NLP?\n"
        "A: First, unsupervised domain adaptation involves transferring knowledge from a labeled source domain "
        "to an unlabeled target domain. Then, the main challenges include domain shift, scarcity of target labels, "
        "and potential overfitting to source data. Finally, researchers address these challenges using adversarial "
        "training, self-supervised methods, and domain-invariant feature extraction.\n\n"
    )

    augmented_query = (
        few_shot_example +
        "The context below is for your reference only.\n\n" +
        f"Context: {context}\n\n" +
        "Question: " + user_query + "\n\n" +
        "### Answer:\n"
    )

    # Set up the text-generation pipeline using the 1B Llama model with adjusted parameters
    generator = pipeline(
        "text-generation",
        model="meta-llama/Llama-3.2-1B-Instruct",
        tokenizer="meta-llama/Llama-3.2-1B-Instruct",
        device=0
    )

    response = generator(
        augmented_query,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

    generated_text = response[0]["generated_text"]

    # Extract only the answer portion after the marker
    if "### Answer:" in generated_text:
        answer = generated_text.split("### Answer:")[-1].strip()
    else:
        answer = generated_text

    # Return both the context and the final answer
    final_output = f"Context:\n{context}\n\nAnswer:\n{answer}"
    return final_output


In [ ]:
import gradio as gr

# Optional: Custom CSS to hide any element with a flag attribute (if one still appears)
custom_css = """
button[aria-label="Flag"] {
  display: none !important;
}
"""

def generate_answer_button(query):
    # Simply call your existing function
    return generate_answer(query)

with gr.Blocks(css=custom_css) as interface:
    # Title and description (optional)
    gr.Markdown("# AI RAG Chatbot")
    gr.Markdown("")

    # Input Textbox
    query_box = gr.Textbox(
        label="How may I assist you today",
        lines=2,
        placeholder="e.g. How to develop a RAG model for LLM"
    )

    # Submit Button
    submit_button = gr.Button("Submit")

    # Output Textbox
    answer_box = gr.Textbox(label="Generated Answer")

    # Link the button click to the function
    submit_button.click(
        fn=generate_answer_button,
        inputs=query_box,
        outputs=answer_box
    )

# Launch the interface
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cceeb1aee507bf9da7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
